In [11]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [12]:
out='../output/'
inb='../input/flows/'
pref='dam'
inp = inb + pref + '/'

In [13]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2016-01-01,160,ZAMZ,0,ZAMC
1,2016-01-01,170,ZIM,0,ZAMZ
2,2016-01-01,160,ZAMZ,1,ZAMC
3,2016-01-01,170,ZIM,1,ZAMZ
4,2016-01-01,160,ZAMZ,2,ZAMC
...,...,...,...,...,...
5121,2021-04-30,33,RSAN,23,BOT
5122,2021-04-30,26,RSAS,23,NAM
5123,2021-04-30,33,ZAMZ,23,ZAMC
5124,2021-04-30,33,ZIM,23,ZAMZ


In [14]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

price  volume  year  month  day  day_of_week      value  \
date       hour                                                             
2016-01-01 0      40.10   477.5  2016      1    1            4  19147.750   
           1      40.08   486.6  2016      1    1            4  19502.928   
           2      40.06   495.6  2016      1    1            4  19853.736   
           3      40.08   496.6  2016      1    1            4  19903.728   
           4      40.07   502.9  2016      1    1            4  20151.203   
...                 ...     ...   ...    ...  ...          ...        ...   
2021-05-31 19    288.09    53.8  2021      5   31            0  15499.242   
           20    118.04    60.0  2021      5   31            0   7082.400   
           21     79.60    73.0  2021      5   31            0   5810.800   
           22     41.15   126.0  2021      5   31            0   5184.900   
           23     41.11    96.0  2021      5   31            0   3946.560   

                      cat  
date       hour            
2016-01-01 0          off  
           1          off  
           2          off  
           3          off  
           4          off  
...                   ...  
2021-05-31 19     evening  
           20    standard  
           21    standard  
           22         off  
           23         off  

[46840 rows x 8 columns]

In [15]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2016-01-01,0,160,ZAMZ,ZAMC,2016,1,1,4,40.10,6416.00
1,2016-01-01,0,170,ZIM,ZAMZ,2016,1,1,4,40.10,6817.00
2,2016-01-01,1,160,ZAMZ,ZAMC,2016,1,1,4,40.08,6412.80
3,2016-01-01,1,170,ZIM,ZAMZ,2016,1,1,4,40.08,6813.60
4,2016-01-01,2,160,ZAMZ,ZAMC,2016,1,1,4,40.06,6409.60
...,...,...,...,...,...,...,...,...,...,...,...
333404,2021-04-30,23,33,RSAN,BOT,2021,4,30,4,44.05,1453.65
333405,2021-04-30,23,26,RSAS,NAM,2021,4,30,4,44.05,1145.30
333406,2021-04-30,23,33,ZAMZ,ZAMC,2021,4,30,4,44.05,1453.65
333407,2021-04-30,23,33,ZIM,ZAMZ,2021,4,30,4,44.05,1453.65


In [16]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2016-01-01,0,160,6416.00,40.10,ZAMC,2016,1,1,4
1,2016-01-01,0,170,6817.00,40.10,ZAMZ,2016,1,1,4
2,2016-01-01,1,160,6412.80,40.08,ZAMC,2016,1,1,4
3,2016-01-01,1,170,6813.60,40.08,ZAMZ,2016,1,1,4
4,2016-01-01,2,160,6409.60,40.06,ZAMC,2016,1,1,4
...,...,...,...,...,...,...,...,...,...,...
333404,2021-04-30,23,-33,-1453.65,44.05,RSAN,2021,4,30,4
333405,2021-04-30,23,-26,-1145.30,44.05,RSAS,2021,4,30,4
333406,2021-04-30,23,-33,-1453.65,44.05,ZAMZ,2021,4,30,4
333407,2021-04-30,23,-33,-1453.65,44.05,ZIM,2021,4,30,4


In [17]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [18]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

node  year  month  day  day_of_week    index  flow    value  \
date       hour                                                                
2016-01-01 0     ZAMC  2016      1    1            4        0   160  6416.00   
           0     ZAMZ  2016      1    1            4        1    10   401.00   
           0      ZIM  2016      1    1            4        1  -170 -6817.00   
           1     ZAMC  2016      1    1            4        2   160  6412.80   
           1     ZAMZ  2016      1    1            4        5    10   400.80   
...               ...   ...    ...  ...          ...      ...   ...      ...   
2021-04-30 23     LES  2021      4   30            4   333401    -3  -132.15   
           23     MOZ  2021      4   30            4   333402   -57 -2510.85   
           23     NAM  2021      4   30            4   333405    26  1145.30   
           23     RSA  2021      4   30            4  2000418     1    44.05   
           23    ZAMC  2021      4   30            4   333406    33  1453.65   

                  price  cat  
date       hour               
2016-01-01 0      40.10  off  
           0      80.20  off  
           0      40.10  off  
           1      40.08  off  
           1      80.16  off  
...                 ...  ...  
2021-04-30 23     44.05  off  
           23     44.05  off  
           23     44.05  off  
           23    264.30  off  
           23     44.05  off  

[196189 rows x 10 columns]

In [19]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

flow_sum     value  hour_count  \
year month node cat      side                                      
2016 1     BOT  evening  supply        -25  -1501.40           2   
                morning  supply       -285 -14572.40           4   
                off      supply      -1270 -59474.14          45   
                standard supply      -1165 -57809.40          18   
           HCB  evening  supply        -57  -6204.59           2   
...                                    ...       ...         ...   
2021 4     ZIM  morning  consume        10    299.90           1   
                         supply        -65  -9768.85           3   
                off      consume      1744  51191.80          28   
                standard consume        52   1557.40           1   
                         supply        -97 -21410.13           5   

                                  price_weight_mean  price_mean      power  
year month node cat      side                                               
2016 1     BOT  evening  supply           60.056000   60.050000 -12.500000  
                morning  supply           51.131228   55.290000 -71.250000  
                off      supply           46.830031   48.463556 -28.222222  
                standard supply           49.621803   56.768333 -64.722222  
           HCB  evening  supply          108.852456   99.140000 -28.500000  
...                                             ...         ...        ...  
2021 4     ZIM  morning  consume          29.990000  149.950000  10.000000  
                         supply          150.290000  740.260000 -21.666667  
                off      consume          29.353096  129.655000  62.285714  
                standard consume          29.950000  149.750000  52.000000  
                         supply          220.722990  675.900000 -19.400000  

[2360 rows x 6 columns]

In [20]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')